In [1]:
import os

from langchain.agents import Tool
from langchain.memory import ChatMessageHistory, ConversationBufferWindowMemory
from langchain_google_genai import (
    ChatGoogleGenerativeAI,
    HarmBlockThreshold,
    HarmCategory,
)
from langchain.utilities import DuckDuckGoSearchAPIWrapper
from langchain.agents import AgentExecutor
from langchain import hub
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import ReActSingleInputOutputParser
from langchain.tools.render import render_text_description

c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from datetime import datetime
hoje = datetime.today()

data_formatada = hoje.strftime("%Y/%m/%d")
# Pega o dia da semana em inglês
dia_da_semana_ingles = hoje.strftime("%A")

# Dicionário para mapear os dias da semana de inglês para português
dias_da_semana_pt = {
    "Monday": "segunda-feira",
    "Tuesday": "terça-feira",
    "Wednesday": "quarta-feira",
    "Thursday": "quinta-feira",
    "Friday": "sexta-feira",
    "Saturday": "sábado",
    "Sunday": "domingo"
}

# Traduz o dia da semana para português
dia_da_semana_pt = dias_da_semana_pt.get(dia_da_semana_ingles, dia_da_semana_ingles)

# Formata a data para utilizar no prompt
data_atual = f"{dia_da_semana_pt}, {data_formatada}"

In [3]:
#(https://makersuite.google.com/)
from dotenv import load_dotenv

load_dotenv()
os.environ["GOOGLE_API_KEY"] = os.getenv('GENERATIVE_LANGUAGE_API_KEY')
os.environ["LANGCHAIN_API_KEY"] = os.getenv('LANGCHAIN_API_KEY')

In [4]:
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    convert_system_message_to_human=True,
    handle_parsing_errors=True,
    temperature=0.6,
    max_tokens= 1000,
    safety_settings = {
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
    },
)

In [5]:
from planing_tools import weatherapi_forecast_periods, query_rag
ddg_search = DuckDuckGoSearchAPIWrapper()

travel_planing_tools = [
    Tool(
        name="DuckDuckGo Search",
        func=ddg_search.run,
        description="""Essa ferramenta DEVE ser utilizada para buscar eventos relevantes no período fornecido pelo usuário. 
        Ela é útil para obter informações sobre eventos ou atividades especiais que estão acontecendo na cidade de destino nas datas que o usuário informou. 
        O modelo deve usá-la para complementar as sugestões de atividades."""
    ),
    Tool(
        name="Weather Forecast",
        func=weatherapi_forecast_periods,
        description="""Essa ferramenta deve ser usada apenas quando o modelo já tiver todas as informações necessárias para gerar o roteiro e o usuário já tiver fornecido as datas exatas. 
        Se o usuário pedir informações para um período de tempo, o modelo deve consultar o clima separadamente para cada dia desse período. 
        Ela deve ser usada quando a previsão do tempo for um fator importante para o planejamento, mas apenas depois que o modelo tiver claro os detalhes do itinerário."""
    ),
    Tool(
        name="Query RAG",
        func=query_rag,
        description="""Esta ferramenta deve ser usada quando o modelo souber a cidade de destino e os interesses do usuário, com o objetivo de fornecer informações sobre pontos turísticos e atrações que se alinham com esses interesses. 
        O modelo deve utilizar essa ferramenta para sugerir atividades e lugares específicos a visitar, baseados na cidade e nos interesses fornecidos."""
    )
]

In [6]:
from calendar_tools import list_calendar_list, list_calendar_events, insert_calendar_event, create_calendar

google_calendar_tools = [
    Tool(
        name="Create Calendar List",
        func=create_calendar,
        description="""
        Use a função create_calendar_list para criar um novo calendário.
        Forneça o resumo (título) do calendário que você deseja criar.
        - Exemplo de uso: 'Meu Calendário de Viagens'
        - Essa função vai criar um novo calendário com o resumo fornecido.
        """
    ),
    Tool(
        name="List Calendar List",
        func=list_calendar_list,
        description="""
        Use a função list_calendar_list para obter a lista dos seus calendários disponíveis no Google Calendar.
        Se você quiser limitar o número de calendários retornados, forneça o número máximo desejado.
        - Exemplo de uso: Forneça o número máximo de calendários a serem listados, como 50. Se não quiser um limite, apenas diga para listar 200 o máximo por padrão.
        """
    ),
    Tool(
        name="List Calendar Events",
        func=list_calendar_events,
        description="""
        Use a função list_calendar_events para obter a lista de eventos de um calendário específico.
        Você precisa fornecer o ID do calendário e, opcionalmente, o número máximo de eventos a serem listados.
        - Exemplo de uso para o calendário principal: forneça 'primary' como o ID do calendário.
        - Exemplo de uso com limite: forneça o ID do calendário e o número máximo de eventos, como 'primary', 20.
        - Para listar eventos de um calendário específico, primeiro liste seus calendários para encontrar o ID correto.
        """
    ),
    Tool(
        name="Insert Calendar Event",
        func=insert_calendar_event,  # Use o novo wrapper aqui
        description="""
        Use a função Insert Calendar Event para adicionar um evento a um calendário.
        Você precisa fornecer um objeto JSON contendo os detalhes do evento.
        Se o timezone não for mencionado utilize "America/Fortaleza"
        Se o usuário informar o nome do calendário para inserir o evento chame a função list_calendar_list primeiro para adquirir a informação de calendar_id
        O objeto JSON deve ter a seguinte estrutura:
        ```json
        {
        "calendar_id": "primary", # Utilize primary 
        "summary": "Dormir",
        "location": "Cama",
        "description": "",
        "start": {
            "dateTime": "2024-12-29T21:00:00",
            "timeZone": "America/Fortaleza"
        },
        "end": {
            "dateTime": "2024-12-29T22:00:00",
            "timeZone": "America/Fortaleza"
        },
        "attendees": []
        }
        ```

        Exemplo de uso: Forneça o objeto JSON completo com os detalhes do evento e o ID do calendário.
        """
    )
]

calendar v3 service created successfully


In [7]:
travel_planing_prompt = hub.pull("tales/agente_turismo")
google_calendar_prompt = hub.pull("tales/agente_calendario")

In [8]:
print(google_calendar_prompt)

input_variables=['agent_scratchpad', 'chat_history', 'data_atual', 'input', 'tool_names', 'tools'] input_types={} partial_variables={} metadata={'lc_hub_owner': 'tales', 'lc_hub_repo': 'agente_calendario', 'lc_hub_commit_hash': '43471ac6b7b30a9dcee36d1e8d2e36315bf430823d5dea0f7190ea60afc27fa5'} template='Você é um agente equipado com diversos funções relacionadas a conta do Calendário Google para ajudar a gerenciar o Calendário Google do usuário.\n\nSempre se lembre de utilizar a sintaxe do Python quando chamar as funções, já que o código utilizou essa linguagem. Por exemplo true DEVE ser True.\n\nHoje é **{data_atual}**. Use essa informação para auxiliar o planejamento de eventos e sugerir ajustes conforme necessário.\n\nTOOLS:\n------\n\nO assistente tem acesso às seguintes ferramentas:\n\n{tools}\n\nPara usar uma ferramenta, siga este formato:\n\n```\nThought: Do I need to use a tool? Yes\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the ac

In [9]:
# planing_prompt = travel_planing_prompt.partial(
#     tools=render_text_description(travel_planing_tools),
#     tool_names=", ".join([t.name for t in travel_planing_tools]),
# )
# llm_with_stop = llm.bind(stop=["\nObservation"])

In [10]:
calendar_prompt = google_calendar_prompt.partial(
    tools=render_text_description(google_calendar_tools),
    tool_names=", ".join([t.name for t in google_calendar_tools]),
)
llm_with_stop = llm.bind(stop=["\nObservation"])

In [11]:
history = ChatMessageHistory()
memory = ConversationBufferWindowMemory(k=20, chat_memory=history, memory_key="chat_history")

C:\Users\Tales\AppData\Local\Temp\ipykernel_19544\275711510.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(k=20, chat_memory=history, memory_key="chat_history")


In [12]:
# travel_planing_agent = (
#     {
#         "input": lambda x: x["input"],
#         "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"]),
#         "chat_history": lambda x: x["chat_history"],
#         "data_atual": lambda x: data_atual,
#     }
#     | planing_prompt
#     | llm_with_stop
#     | ReActSingleInputOutputParser()
# )

# travel_agent_executor = AgentExecutor(agent=travel_planing_agent, tools=travel_planing_tools, verbose=True, memory=memory)

In [13]:
google_calendar_agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"]),
        "chat_history": lambda x: x["chat_history"],
        "data_atual": lambda x: data_atual,
    }
    | calendar_prompt
    | llm_with_stop
    | ReActSingleInputOutputParser()
)

calendar_agent_executor = AgentExecutor(agent=google_calendar_agent, tools=google_calendar_tools, verbose=True, memory=memory)

In [14]:
response = calendar_agent_executor.invoke({"input": f"""
                                  Marque no meu calendário Teste-29-12 para hoje às 18:00h, um compromisso chamado 'Consulta Médica' no 'Hospital Central'
                                  """})

print(response["output"])



> Entering new AgentExecutor chain...


c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


```tool_code
Thought: Do I need to use a tool? Yes
Action: List Calendar List
Action Input: 200[{'id': 'addressbook#contacts@group.v.calendar.google.com', 'name': 'Aniversários', 'description': 'Mostra aniversários, datas comemorativas e outros eventos de pessoas no Contatos do Google.'}, {'id': '23a8bf709131ca92793738f86020bf528f56e15a5cf122b6827df2b5c46b7fbc@group.calendar.google.com', 'name': 'Teste-29-12', 'description': ''}, {'id': 'tales.v.m.alves@gmail.com', 'name': 'tales.v.m.alves@gmail.com', 'description': ''}, {'id': 'pt-br.brazilian#holiday@group.v.calendar.google.com', 'name': 'Feriados no Brasil', 'description': 'Feriados e datas comemorativas no Brasil'}]

c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


```tool_code
Do I need to use a tool? Yes
Action: Insert Calendar Event
Action Input: {
  "calendar_id": "23a8bf709131ca92793738f86020bf528f56e15a5cf122b6827df2b5c46b7fbc@group.calendar.google.com",
  "summary": "Consulta Médica",
  "location": "Hospital Central",
  "description": "",
  "start": {
    "dateTime": "2024-12-29T18:00:00",
    "timeZone": "America/Fortaleza"
  },
  "end": {
    "dateTime": "2024-12-29T19:00:00",
    "timeZone": "America/Fortaleza"
  },
  "attendees": []
}{'kind': 'calendar#event', 'etag': '"3471020994168000"', 'id': '1hs4jor9drj0o6h56uriul18ok', 'status': 'confirmed', 'htmlLink': 'https://www.google.com/calendar/event?eid=MWhzNGpvcjlkcmowbzZoNTZ1cml1bDE4b2sgMjNhOGJmNzA5MTMxY2E5Mjc5MzczOGY4NjAyMGJmNTI4ZjU2ZTE1YTVjZjEyMmI2ODI3ZGYyYjVjNDZiN2ZiY0Bn', 'created': '2024-12-29T22:14:57.000Z', 'updated': '2024-12-29T22:14:57.084Z', 'summary': 'Consulta Médica', 'location': 'Hospital Central', 'creator': {'email': 'tales.v.m.alves@gmail.com'}, 'organizer': {'email':

c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


```tool_code
Do I need to use a tool? No
Final Answer: Evento "Consulta Médica" criado no calendário "Teste-29-12" para hoje, 29/12/2024, às 18:00 no Hospital Central.
```


> Finished chain.
Evento "Consulta Médica" criado no calendário "Teste-29-12" para hoje, 29/12/2024, às 18:00 no Hospital Central.
```


In [14]:
response = calendar_agent_executor.invoke({"input": f"""
                                  Crie um calendário chamado Teste-29-12
                                  """})

print(response["output"])



> Entering new AgentExecutor chain...


c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Thought: Do I need to use a tool? Yes
Action: Create Calendar List
Action Input: 'Teste-29-12'{'kind': 'calendar#calendar', 'etag': '"3jxDgHTXkTGCqi8rfpu1SVSANnY"', 'id': '23a8bf709131ca92793738f86020bf528f56e15a5cf122b6827df2b5c46b7fbc@group.calendar.google.com', 'summary': 'Teste-29-12', 'timeZone': 'UTC', 'conferenceProperties': {'allowedConferenceSolutionTypes': ['hangoutsMeet']}}

c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Do I need to use a tool? No
Final Answer: Calendário "Teste-29-12" criado com sucesso! O ID do calendário é: `23a8bf709131ca92793738f86020bf528f56e15a5cf122b6827df2b5c46b7fbc@group.calendar.google.com`.  Você pode agora adicionar eventos a ele usando a função `Insert Calendar Event`.


> Finished chain.
Calendário "Teste-29-12" criado com sucesso! O ID do calendário é: `23a8bf709131ca92793738f86020bf528f56e15a5cf122b6827df2b5c46b7fbc@group.calendar.google.com`.  Você pode agora adicionar eventos a ele usando a função `Insert Calendar Event`.


In [15]:
response = calendar_agent_executor.invoke({"input": f"""
                                  Liste os meus calendários
                                  """})

print(response["output"])



> Entering new AgentExecutor chain...


c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


```tool_code
Thought: Do I need to use a tool? Yes
Action: List Calendar List
Action Input: 200[{'id': 'addressbook#contacts@group.v.calendar.google.com', 'name': 'Aniversários', 'description': 'Mostra aniversários, datas comemorativas e outros eventos de pessoas no Contatos do Google.'}, {'id': 'pt-br.brazilian#holiday@group.v.calendar.google.com', 'name': 'Feriados no Brasil', 'description': 'Feriados e datas comemorativas no Brasil'}, {'id': 'tales.v.m.alves@gmail.com', 'name': 'tales.v.m.alves@gmail.com', 'description': ''}, {'id': 'ed44b535d9e3b857a66c5f89cc05bef36c9ebf9d4d5256f00f16d3acc95641cd@group.calendar.google.com', 'name': "'Teste-29-12'", 'description': ''}, {'id': '23a8bf709131ca92793738f86020bf528f56e15a5cf122b6827df2b5c46b7fbc@group.calendar.google.com', 'name': 'Teste-29-12', 'description': ''}]

c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


```tool_code
Do I need to use a tool? No
Final Answer: Seus calendários são: Aniversários, Feriados no Brasil, tales.v.m.alves@gmail.com, 'Teste-29-12', e Teste-29-12.  Note que "Teste-29-12" aparece duas vezes, possivelmente devido a um erro de criação ou duplicação.  Você pode querer verificar e remover a duplicata.
```


> Finished chain.
Seus calendários são: Aniversários, Feriados no Brasil, tales.v.m.alves@gmail.com, 'Teste-29-12', e Teste-29-12.  Note que "Teste-29-12" aparece duas vezes, possivelmente devido a um erro de criação ou duplicação.  Você pode querer verificar e remover a duplicata.
```


In [ ]:
response = travel_agent_executor.invoke({"input": f"""
                                  Boa noite
                                  """})

print(response["output"])

In [ ]:
response = travel_agent_executor.invoke({"input": f"""
                                  Eu estou planejando visitar Natal no Rio Grande do Norte
                                  """})

print(response["output"])

In [ ]:
response = travel_agent_executor.invoke({"input": f"""
                                  Eu planejo passar da quarta dia 01 de janeiro até o domingo
                                  """})

print(response["output"])

In [ ]:
response = travel_agent_executor.invoke({"input": f"""
                                  Eu planejo visitar praias e restaurantes regionais
                                  """})

print(response["output"])

In [ ]:
response = travel_agent_executor.invoke({"input": f"""
                                  Sem incluir Hóteis pretendo gastar no máximo 600 reais
                                  """})

print(response["output"])

In [ ]:
response = travel_agent_executor.invoke({"input": f"""
                                  Eu não possuo nenhuma alergia
                                  """})

print(response["output"])

In [ ]:
response = travel_agent_executor.invoke({"input": f"""
                                  Agitada.
                                  """})

print(response["output"])

In [ ]:
print(response["chat_history"])